In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}?charset=utf8mb4"
                       .format(user="openalex_ym",
                               pw="openalex.ym",
                               host="172.18.18.167",
                               port=3306,
                               db="openalex2022"),
                       )

In [2]:
target_author = pd.read_csv('target_author.csv')

In [3]:
target_author.head(3)

,author_id,MAGAuthorID,AFTMentorID,FirstMenteeYear
0,A367921,367921,2488,2000
1,A980010,980010,12407,1991
2,A1121383,1121383,11355,2009


In [4]:
len(target_author)

29011

In [6]:
auid = list(target_author.author_id)
print(len(auid))

29011


##### Find Target Author's Papers

In [7]:
author_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = auid[start_point:end_point]
        author_groups.append(temp)
    except:
        temp = auid[start_point:]
        author_groups.append(temp)
        
    start_point = end_point
    if end_point>len(auid):
        break

In [8]:
l = len(author_groups)
for i in range(l):
    temp =  author_groups[i]
    sql_query = """select works_authorships.author_id, works_authorships.work_id

                    from works_authorships 

                    where works_authorships.author_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))

data = tp

In [9]:
data.head(3)

,author_id,work_id
0,A100047927,W2531684035
1,A100047927,W2596839079
2,A100047927,W2694607381


#### Find full authorships of the target authors' papers

In [10]:
paper = list(set(data.work_id))
print(len(paper))

2730484


In [11]:
paper_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = paper[start_point:end_point]
        paper_groups.append(temp)
    except:
        temp = paper[start_point:]
        paper_groups.append(temp)
        
    start_point = end_point
    if end_point>len(paper):
        break

In [12]:
len(paper_groups)

547

In [13]:
l = len(paper_groups)
for i in range(l):
    temp =  paper_groups[i]
    sql_query = """select *

                    from works_authorships 

                    where works_authorships.work_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    print(i)

Data = tp

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
len(Data)

14288750

In [15]:
Data.head(5)

,work_id,author_position,author_id,institution_id
0,W1000433918,first,A2236851489,I1322918889\r
1,W1000433918,middle,A2443213595,I2613432\r
2,W1000433918,middle,A2301774941,I1322918889\r
3,W1000433918,middle,A2202964799,I204465549\r
4,W1000433918,last,A2264557872,I204465549\r


In [16]:
inst_ids = list(Data.institution_id)
inst_ids1 = [k.replace('\r', '') for k in inst_ids]

In [17]:
Data['institution_id'] = inst_ids1

In [18]:
Data.head(5)

,work_id,author_position,author_id,institution_id
0,W1000433918,first,A2236851489,I1322918889
1,W1000433918,middle,A2443213595,I2613432
2,W1000433918,middle,A2301774941,I1322918889
3,W1000433918,middle,A2202964799,I204465549
4,W1000433918,last,A2264557872,I204465549


In [19]:
Data.to_csv('work_authorship.csv')